# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# CSV path created in Part A
updated_city_data_csv = "../WeatherPy/output_data/updated_city_data.csv"

# Create df for cities
updated_city_data = pd.read_csv(updated_city_data_csv)
updated_city_data.head(5)

,City_ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,0,champerico,70.0,GT,1.665344e+09,72.0,14.3000,-91.9167,90.14,4.14
1,1,hilo,20.0,US,1.665344e+09,67.0,19.7297,-155.0900,87.28,5.01
2,2,yellowknife,75.0,CA,1.665344e+09,87.0,62.4560,-114.3525,46.92,15.01
3,3,kapaa,0.0,US,1.665344e+09,67.0,22.0752,-159.3190,81.10,1.01
4,4,bluff,100.0,NZ,1.665344e+09,75.0,-46.6000,168.3333,50.14,11.27


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Use .configure to configure google API key
gmaps.configure(api_key = g_key)

In [4]:
# Drop NA values
updated_city_data = updated_city_data.dropna()

# Add Latitude & Longitude = Locations 
locations = updated_city_data[["Latitude", "Longitude"]]
humidity = updated_city_data["Humidity"]

# Add the Humidity heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100, 
                                 point_radius = 2)

fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
# Narrow down the DataFrame to find your ideal weather condition.
ideal_weather_df = updated_city_data.loc[(city_data_df["Max Temp"] < 80) 
                                    & (city_data_df["Max Temp"] > 70) 
                                    & (city_data_df["Wind Speed"] < 10) 
                                    & (city_data_df["Cloudiness"] == 0),:].dropna()
# Drop Date column before preview
ideal_weather_df = ideal_weather_df.drop('Date', axis=1)
ideal_weather_df.head(5)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df = ideal_weather_df[["City", "Country", "Latitude", "Longitude"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(5)

,City,Country,Latitude,Longitude,Hotel Name
83,amboasary,MG,-25.0333,46.3833,
113,montepuez,MZ,-13.1256,38.9997,
132,wilmington,US,34.2257,-77.9447,
135,waddan,LY,29.1614,16.1390,
150,gat,IL,31.6100,34.7642,


In [16]:
# Set parameters to serach for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through cities
for index, row in hotel_df.iterrows():
    # get lat, lng from dataframe
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while keeping original params in place
    params["location"] = f"{lat},{lng}"
    
    # Create base url to search nearby hotel     
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request
    print(f"Retrieving results for city: {row['City']}")
    hotel_name = requests.get(base_url, params=params).json()
    
    # extract results
    hotel_name = hotel_name['results']
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name[0]['name']
        print(f"Closest hotel is {hotel_name[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = hotel_name[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

# Display hotel names on dataframe
hotel_df.head(5)

Retrieving results for city: amboasary
Closest hotel is Le Bon Samaritain.
------------
Retrieving results for city: montepuez
Closest hotel is PembaAparthotel.
------------
Retrieving results for city: wilmington
Closest hotel is Hotel Ballast Wilmington, Tapestry Collection by Hilton.
------------
Retrieving results for city: waddan
Closest hotel is محل الشريف محمد العايب.
------------
Retrieving results for city: gat
Closest hotel is OlusHome.
------------
Retrieving results for city: inhambane
Closest hotel is Oceano.
------------
Retrieving results for city: alcudia
Closest hotel is Zafiro Tropic.
------------
Retrieving results for city: harper
Closest hotel is Screensaver.
------------
Retrieving results for city: senanga
Closest hotel is Zango lodge.
------------
Retrieving results for city: junin
Closest hotel is Hotel Copahue.
------------
Retrieving results for city: shakawe
Closest hotel is EdenWood Lodge.
------------
Retrieving results for city: lobatse
Closest hotel is S

,City,Country,Latitude,Longitude,Hotel Name
83,amboasary,MG,-25.0333,46.3833,Le Bon Samaritain
113,montepuez,MZ,-13.1256,38.9997,PembaAparthotel
132,wilmington,US,34.2257,-77.9447,"Hotel Ballast Wilmington, Tapestry Collection ..."
135,waddan,LY,29.1614,16.1390,محل الشريف محمد العايب
150,gat,IL,31.6100,34.7642,OlusHome


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [12]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))